<h1><center>🧩Jigsaw🧩: All Competition EDA at One Place</center></h1>
                                                      
<center><img src = "https://jigsaw.google.com/static/images/social-share.jpg?cache=df11f5c" width = "750" height = "500"/></center>                                                                            

<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Contents</center></h2>

1. [Competition Overview](#competition-overview)    
2. [Libraries](#libraries)  
3. [Weights and Biases](#weights-and-biases)  
4. [Global Config](#global-config)  
5. [Load Datasets](#load-datasets) 
6. [Tabular Exploration](#tabular-exploration)
7. [Standard NLP Exploration](#standard-nlp-exploration)
8. [Toxicity and Polarity](#toxicity-and-polarity)
9. [Model Config](#model-config)
10. [Utilities](#utilities)
11. [Dataset Class](#dataset-class)
12. [Model](#model)
13. [Engine](#engine)
14. [Run](#run)
14. [References](#references)

<div class="list-group" id="list-tab" role="tablist">
<h3 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:white' role="tab" aria-controls="home"><center>If you find this notebook useful, do give me an upvote, it helps to keep up my motivation. This notebook will be updated frequently so keep checking for furthur developments.</center></h3>

<a id="competition-overview"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Competition Overview</center></h2>

## **<span style="color:orange;">Description</span>**

In this competition, we will be asking you to score a set of about fourteen thousand comments. Pairs of comments were presented to expert raters, who marked one of two comments more harmful — each according to their own notion of toxicity. In this contest, when you provide scores for comments, they will be compared with several hundred thousand rankings. Your average agreement with the raters will determine your individual score. In this way, we hope to focus on ranking the severity of comment toxicity from innocuous to outrageous, where the middle matters as much as the extremes.

## **<span style="color:orange;">Evaluation Criteria</span>**

Submissions are evaluated on Average Agreement with Annotators. For the ground truth, annotators were shown two comments and asked to identify which of the two was more toxic. Pairs of comments can be, and often are, rated by more than one annotator, and may have been ordered differently by different annotators.
  
For each of the approximately 200,000 pair ratings in the ground truth test data, we use your predicted toxicity score to rank the comment pair. The pair receives a 1 if this ranking matches the annotator ranking, or 0 if it does not match.
  
The final score is the average across all the pair evaluations.
  
Please note the following:
  
- score is not constrained to any numeric range (e.g., you can predict [0, 1] or [-999, 999]).
- There is no tie breaking; tied comment scores will always be evaluated as 0. You could consider using something like scipy.stats.rankdata to force unique value.

<a id="libraries"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Libraries</center></h2>

In [ ]:
import gc
import os
import pdb
import time
import glob
import sys
import math
import random
import wandb
import math

import numpy as np
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.patches as patches

plt.rcParams.update({'font.size': 18})
plt.style.use('fivethirtyeight')

import seaborn as sns
import matplotlib
from termcolor import colored

import warnings
warnings.filterwarnings("ignore")

import re
import folium
# import textstat
from scipy import stats
from colorama import Fore, Back, Style, init
from bs4 import BeautifulSoup

import scipy as sp

import random
import networkx as nx
from pandas import Timestamp
from collections import defaultdict

from PIL import Image
from IPython.display import SVG

import requests
from IPython.display import HTML

import matplotlib.cm as cm

from tqdm import tqdm
tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import transformers
from sklearn.metrics import accuracy_score, roc_auc_score
from tokenizers import BertWordPieceTokenizer

from sklearn import metrics
from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer,\
                                            CountVectorizer,\
                                            HashingVectorizer

from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer  

import nltk
from textblob import TextBlob

from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
# from polyglot.detect import Detector
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import OneCycleLR
    
# NLP
from transformers import AutoTokenizer, AutoModel

stopword=set(STOPWORDS)

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

In [ ]:
import wandb
wandb.login()

<a id="weights-and-biases"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Weights and Biases (W&B)</center></h2>

<center><img src = "https://i.imgur.com/1sm6x8P.png" width = "750" height = "500"/></center>  

**Weights & Biases** is the machine learning platform for developers to build better models faster. 

You can use W&B's lightweight, interoperable tools to 
- quickly track experiments, 
- version and iterate on datasets, 
- evaluate model performance, 
- reproduce models, 
- visualize results and spot regressions, 
- and share findings with colleagues. 

Set up W&B in 5 minutes, then quickly iterate on your machine learning pipeline with the confidence that your datasets and models are tracked and versioned in a reliable system of record.

In this notebook I will use Weights and Biases's amazing features to perform wonderful visualizations and logging seamlessly. 

<a id="global-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Global Config</center></h2>

In [ ]:
class config:
    DIRECTORY_PATH = "/kaggle/input/jigsaw-multilingual-toxic-comment-classification"
    TRAIN_PATH = DIRECTORY_PATH + "/jigsaw-toxic-comment-train.csv"
    
# Device Optimization
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [ ]:
# wandb config
WANDB_CONFIG = {
     'competition': 'Jigsaw', 
              '_wandb_kernel': 'neuracort'
    }

<a id="load-datasets"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Load Datasets</center></h2>

In [ ]:
train = pd.read_csv(config.TRAIN_PATH)

<a id="tabular-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Tabular Exploration</center></h2>

## **<span style="color:orange;">Dataset Head and Info</span>**

Let us quickly have a brief look upon the dataset. Hope to not encounter any NULL values!

In [ ]:
train.head()

In [ ]:
train.info()

Aha! There are no NULL values, so we are good to go.

## **<span style="color:orange;">Dataset Size</span>**

In [ ]:
print(f"Training Dataset Shape: {colored(train.shape, 'yellow')}")

## **<span style="color:orange;">Column-wise Unique Values</span>**

In [ ]:
for col in train.columns:
    print(col + ":" + colored(str(len(train[col].unique())), 'yellow'))

<a id="standard-nlp-exploration"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Standard NLP Exploration</center></h2>

## **<span style="color:orange;">Wordcloud</span>**

In [ ]:
def nonan(x):
    if type(x) == str:
        return x.replace("\n", "")
    else:
        return ""

text = ' '.join([nonan(abstract) for abstract in train["comment_text"]])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(text)
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Common words in comments')

## **<span style="color:orange;">English vs Non English Comments</span>**

In [ ]:
def get_language(text):
    return Detector("".join(x for x in text if x.isprintable()), quiet=True).languages[0].name

train["lang"] = train["comment_text"].progress_apply(get_language)

In [ ]:
lang_list = sorted(list(set(train["lang"])))
counts = [list(train["lang"]).count(cont) for cont in lang_list]
df = pd.DataFrame(np.transpose([lang_list, counts]))
df.columns = ["Language", "Count"]
df["Count"] = df["Count"].apply(int)

df_en = pd.DataFrame(np.transpose([["English", "Non-English"], [max(counts), sum(counts) - max(counts)]]))
df_en.columns = ["Language", "Count"]

fig = px.bar(df_en, x="Language", y="Count", title="Language of comments", color="Language", text="Count")
fig.update_layout(template="plotly_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig.data[0].textfont.color = "black"
fig.data[0].textposition = "outside"
fig.data[1].textfont.color = "black"
fig.data[1].textposition = "outside"
fig

## **<span style="color:orange;">Bar chart of non-English languages</span>**

In [ ]:
fig = px.bar(df.query("Language != 'English' and Language != 'un'").query("Count >= 50"),
             y="Language", x="Count", title="Language of non-English comments", template="plotly_white", color="Language", text="Count", orientation="h")
fig.update_traces(marker=dict(line=dict(width=0.75,
                                        color='black')),  textposition="outside")
fig.update_layout(showlegend=False)
fig

## **<span style="color:orange;">Comment Word Distribution</span>**

In [ ]:
def new_len(x):
    if type(x) is str:
        return len(x.split())
    else:
        return 0

train["comment_words"] = train["comment_text"].apply(new_len)
nums = train.query("comment_words != 0 and comment_words < 200").sample(frac=0.1)["comment_words"]
fig = ff.create_distplot(hist_data=[nums],
                         group_labels=["All comments"],
                         colors=["coral"])

fig.update_layout(title_text="Comment words", xaxis_title="Comment words", template="simple_white", showlegend=False)
fig.show()

## **<span style="color:orange;">Average Comment Words vs Language</span>**

In [ ]:
df = pd.DataFrame(np.transpose([lang_list, train.groupby("lang").mean()["comment_words"]]))
df.columns = ["Language", "Average_comment_words"]
df["Average_comment_words"] = df["Average_comment_words"].apply(float)
df = df.query("Average_comment_words < 500")
fig = go.Figure(go.Bar(x=df["Language"], y=df["Average_comment_words"]))

fig.update_layout(xaxis_title="Language", yaxis_title="Average comment words", title_text="Average comment words vs. language", template="plotly_white")
fig.show()

<a id="toxicity-and-polarity"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Toxicity and Polarity</center></h2>

In [ ]:
def polarity(x):
    if type(x) == str:
        return SIA.polarity_scores(x)
    else:
        return 1000
    
SIA = SentimentIntensityAnalyzer()
train["polarity"] = train["comment_text"].progress_apply(polarity)

In [ ]:
fig = go.Figure(go.Histogram(x=[pols["neg"] for pols in train["polarity"] if pols["neg"] != 0], marker=dict(
            color='seagreen')
    ))

fig.update_layout(xaxis_title="Negativity sentiment", title_text="Negativity sentiment", template="simple_white")
fig.show()

<a id="model-config"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Model Config</center></h2>

In [ ]:
data_dir = '../input/jrstc-train-folds'
train_file_path = os.path.join(data_dir, 'validation_data_5_folds.csv')
train_df = pd.read_csv(train_file_path)

params = {
    'device': device,
    'debug': False,
    'checkpoint': 'roberta-base',
    'output_logits': 768,
    'max_len': 256,
    'num_folds': 1,
    'batch_size': 16,
    'dropout': 0.2,
    'num_workers': 2,
    'epochs': 1,
    'lr': 2e-5,
    'margin': 0.7,
    'scheduler_name': 'OneCycleLR',
    'max_lr': 5e-5,                 # OneCycleLR
    'pct_start': 0.1,               # OneCycleLR
    'anneal_strategy': 'cos',       # OneCycleLR
    'div_factor': 1e3,              # OneCycleLR
    'final_div_factor': 1e3,        # OneCycleLR
    'no_decay': True
}

if params['debug']:
    train_df = train_df.sample(frac=0.01)
    print('Reduced training Data Size for Debugging purposes')

<a id="utilities"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Utilities</center></h2>

## **<span style="color:orange;">Text Preprocessing</span>**

In [ ]:
def text_cleaning(text):
    '''
    Cleans text into a basic form for NLP. Operations include the following:-
    1. Remove special charecters like &, #, etc
    2. Removes extra spaces
    3. Removes embedded URL links
    4. Removes HTML tags
    5. Removes emojis
    
    text - Text piece to be cleaned.
    '''
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip() # remove spaces at the beginning and at the end of string

    return text

tqdm.pandas()
train_df['less_toxic'] = train_df['less_toxic'].progress_apply(text_cleaning)
train_df['more_toxic'] = train_df['more_toxic'].progress_apply(text_cleaning)

## **<span style="color:orange;">Metric Monitor</span>**

In [ ]:
class MetricMonitor:
    def __init__(self, float_precision=4):
        self.float_precision = float_precision
        self.reset()

    def reset(self):
        self.metrics = defaultdict(lambda: {"val": 0, "count": 0, "avg": 0})

    def update(self, metric_name, val):
        metric = self.metrics[metric_name]

        metric["val"] += val
        metric["count"] += 1
        metric["avg"] = metric["val"] / metric["count"]

    def __str__(self):
        return " | ".join(
            [
                "{metric_name}: {avg:.{float_precision}f}".format(
                    metric_name=metric_name, avg=metric["avg"],
                    float_precision=self.float_precision
                )
                for (metric_name, metric) in self.metrics.items()
            ]
        )

## **<span style="color:orange;">Scheduler</span>**

In [ ]:
def get_scheduler(optimizer, scheduler_params=params):
    if scheduler_params['scheduler_name'] == 'CosineAnnealingWarmRestarts':
        scheduler = CosineAnnealingWarmRestarts(
            optimizer,
            T_0=scheduler_params['T_0'],
            eta_min=scheduler_params['min_lr'],
            last_epoch=-1
        )
    elif scheduler_params['scheduler_name'] == 'OneCycleLR':
        scheduler = OneCycleLR(
            optimizer,
            max_lr=scheduler_params['max_lr'],
            steps_per_epoch=int(df_train.shape[0] / params['batch_size']) + 1,
            epochs=scheduler_params['epochs'],
            pct_start=scheduler_params['pct_start'],
            anneal_strategy=scheduler_params['anneal_strategy'],
            div_factor=scheduler_params['div_factor'],
            final_div_factor=scheduler_params['final_div_factor'],
        )
    return scheduler

<a id="dataset-class"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Dataset Class</center></h2>

In [ ]:
class BERTDataset:
    def __init__(self, more_toxic, less_toxic, max_len=params['max_len'], checkpoint=params['checkpoint']):
        self.more_toxic = more_toxic
        self.less_toxic = less_toxic
        self.max_len = max_len
        self.checkpoint = checkpoint
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.num_examples = len(self.more_toxic)

    def __len__(self):
        return self.num_examples

    def __getitem__(self, idx):
        more_toxic = str(self.more_toxic[idx])
        less_toxic = str(self.less_toxic[idx])

        tokenized_more_toxic = self.tokenizer(
            more_toxic,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_attention_mask=True,
            return_token_type_ids=True,
        )

        tokenized_less_toxic = self.tokenizer(
            less_toxic,
            add_special_tokens=True,
            truncation=True,
            padding='max_length',
            max_length=self.max_len,
            return_attention_mask=True,
            return_token_type_ids=True,
        )

        ids_more_toxic = tokenized_more_toxic['input_ids']
        mask_more_toxic = tokenized_more_toxic['attention_mask']
        token_type_ids_more_toxic = tokenized_more_toxic['token_type_ids']

        ids_less_toxic = tokenized_less_toxic['input_ids']
        mask_less_toxic = tokenized_less_toxic['attention_mask']
        token_type_ids_less_toxic = tokenized_less_toxic['token_type_ids']

        return {'ids_more_toxic': torch.tensor(ids_more_toxic, dtype=torch.long),
                'mask_more_toxic': torch.tensor(mask_more_toxic, dtype=torch.long),
                'token_type_ids_more_toxic': torch.tensor(token_type_ids_more_toxic, dtype=torch.long),
                'ids_less_toxic': torch.tensor(ids_less_toxic, dtype=torch.long),
                'mask_less_toxic': torch.tensor(mask_less_toxic, dtype=torch.long),
                'token_type_ids_less_toxic': torch.tensor(token_type_ids_less_toxic, dtype=torch.long),
                'target': torch.tensor(1, dtype=torch.float)}

<a id="model"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Model</center></h2>

In [ ]:
class ToxicityModel(nn.Module):
    def __init__(self, checkpoint=params['checkpoint'], params=params):
        super(ToxicityModel, self).__init__()
        self.checkpoint = checkpoint
        self.bert = AutoModel.from_pretrained(checkpoint, return_dict=False)
        self.layer_norm = nn.LayerNorm(params['output_logits'])
        self.dropout = nn.Dropout(params['dropout'])
        self.dense = nn.Sequential(
            nn.Linear(params['output_logits'], 128),
            nn.SiLU(),
            nn.Dropout(params['dropout']),
            nn.Linear(128, 1)
        )

    def forward(self, input_ids, token_type_ids, attention_mask):
        _, pooled_output = self.bert(input_ids=input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        pooled_output = self.layer_norm(pooled_output)
        pooled_output = self.dropout(pooled_output)
        preds = self.dense(pooled_output)
        return preds

<a id="engine"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Engine</center></h2>

## **<span style="color:orange;">Training Function</span>**

In [ ]:
def train_fn(train_loader, model, criterion, optimizer, epoch, params, scheduler=None):
    metric_monitor = MetricMonitor()
    model.train()
    stream = tqdm(train_loader)
    
    for i, batch in enumerate(stream, start=1):
        ids_more_toxic = batch['ids_more_toxic'].to(device)
        mask_more_toxic = batch['mask_more_toxic'].to(device)
        token_type_ids_more_toxic = batch['token_type_ids_more_toxic'].to(device)
        
        ids_less_toxic = batch['ids_less_toxic'].to(device)
        mask_less_toxic = batch['mask_less_toxic'].to(device)
        token_type_ids_less_toxic = batch['token_type_ids_less_toxic'].to(device)
        
        target = batch['target'].to(device)

        logits_more_toxic = model(ids_more_toxic, token_type_ids_more_toxic, mask_more_toxic)
        logits_less_toxic = model(ids_less_toxic, token_type_ids_less_toxic, mask_less_toxic)
        
        loss = criterion(logits_more_toxic, logits_less_toxic, target)
        
        metric_monitor.update('Loss', loss.item())
        
        loss.backward()
        optimizer.step()
            
        if scheduler is not None:
            scheduler.step()
        
        optimizer.zero_grad()
        stream.set_description(f"Epoch: {epoch:02}. Train. {metric_monitor}")

## **<span style="color:orange;">Validation Function</span>**

In [ ]:
def validate_fn(val_loader, model, criterion, epoch, params):
    metric_monitor = MetricMonitor()
    model.eval()
    stream = tqdm(val_loader)
    all_loss = []
    with torch.no_grad():
        for i, batch in enumerate(stream, start=1):
            ids_more_toxic = batch['ids_more_toxic'].to(device)
            mask_more_toxic = batch['mask_more_toxic'].to(device)
            token_type_ids_more_toxic = batch['token_type_ids_more_toxic'].to(device)
            
            ids_less_toxic = batch['ids_less_toxic'].to(device)
            mask_less_toxic = batch['mask_less_toxic'].to(device)
            token_type_ids_less_toxic = batch['token_type_ids_less_toxic'].to(device)
            
            target = batch['target'].to(device)

            logits_more_toxic = model(ids_more_toxic, token_type_ids_more_toxic, mask_more_toxic)
            logits_less_toxic = model(ids_less_toxic, token_type_ids_less_toxic, mask_less_toxic)
            
            loss = criterion(logits_more_toxic, logits_less_toxic, target)
            all_loss.append(loss.item())
            
            metric_monitor.update('Loss', loss.item())
            
            stream.set_description(f"Epoch: {epoch:02}. Valid. {metric_monitor}")
            
    return np.mean(all_loss)

<a id="run"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>Run</center></h2>

In [ ]:
best_models_of_each_fold = []

In [ ]:
gc.collect()
for fold in range(params['num_folds']):
    print(f'******************** Training Fold: {fold+1} ********************')
    current_fold = fold
    df_train = train_df[train_df['kfold'] != current_fold].copy()
    df_valid = train_df[train_df['kfold'] == current_fold].copy()

    train_dataset = BERTDataset(
        df_train.more_toxic.values,
        df_train.less_toxic.values
    )
    valid_dataset = BERTDataset(
        df_valid.more_toxic.values,
        df_valid.less_toxic.values
    )

    train_dataloader = DataLoader(
        train_dataset, batch_size=params['batch_size'], shuffle=True,
        num_workers=params['num_workers'], pin_memory=True
    )
    valid_dataloader = DataLoader(
        valid_dataset, batch_size=params['batch_size']*2, shuffle=False,
        num_workers=params['num_workers'], pin_memory=True
    )
    
    model = ToxicityModel()
    model = model.to(params['device'])
    criterion = nn.MarginRankingLoss(margin=params['margin'])
    if params['no_decay']:
        param_optimizer = list(model.named_parameters())
        no_decay = ['bias', 'LayerNorm.weight', 'LayerNorm.bias']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
            {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
        optimizer = optim.AdamW(optimizer_grouped_parameters, lr=params['lr'])
    else:
        optimizer = optim.AdamW(model.parameters(), lr=params['lr'])
    scheduler = get_scheduler(optimizer)

    # Training and Validation Loop
    best_loss = np.inf
    best_epoch = 0
    best_model_name = None
    
    # Initialize W&B
    run = wandb.init(project='Jigsaw-RoBerta', config= WANDB_CONFIG)
    
    for epoch in range(1, params['epochs'] + 1):
        train_fn(train_dataloader, model, criterion, optimizer, epoch, params, scheduler)
        valid_loss = validate_fn(valid_dataloader, model, criterion, epoch, params)
        if valid_loss <= best_loss:
            best_loss = valid_loss
            best_epoch = epoch
            if best_model_name is not None:
                os.remove(best_model_name)
            torch.save(model.state_dict(), f"{params['checkpoint']}_{epoch}_epoch_f{fold+1}.pth")
            best_model_name = f"{params['checkpoint']}_{epoch}_epoch_f{fold+1}.pth"
            
        wandb.log({'valid_loss': valid_loss})
        
    # Close W&B run
    wandb.finish()

    # Print summary of this fold
    print('')
    print(f'The best LOSS: {best_loss} for fold {fold+1} was achieved on epoch: {best_epoch}.')
    print(f'The Best saved model is: {best_model_name}')
    best_models_of_each_fold.append(best_model_name)
    del df_train, df_valid, train_dataset, valid_dataset, train_dataloader, valid_dataloader, model
    _ = gc.collect()
    torch.cuda.empty_cache()

<a id="references"></a>
<div class="list-group" id="list-tab" role="tablist">
<h2 class="list-group-item list-group-item-action active" data-toggle="list" style='background:orange; border:0; color:white' role="tab" aria-controls="home"><center>References</center></h2>

> - [Jigsaw Multilingual Toxicity : EDA + Models](https://www.kaggle.com/tarunpaparaju/jigsaw-multilingual-toxicity-eda-models)
> - [Pytorch RoBERTa Ranking Baseline JRSTC [Train]](https://www.kaggle.com/manabendrarout/pytorch-roberta-ranking-baseline-jrstc-train)
>
>---

<h1><center>More Plots and Models coming soon!</center></h1>

<center><img src = "https://static.wixstatic.com/media/5f8fae_7581e21a24a1483085024f88b0949a9d~mv2.jpg/v1/fill/w_934,h_379,al_c,q_90/5f8fae_7581e21a24a1483085024f88b0949a9d~mv2.jpg" width = "750" height = "500"/></center> 

--- 

## **<span style="color:orange;">Let's have a Talk!</span>**
> ### Reach out to me on [LinkedIn](https://www.linkedin.com/in/ishandutta0098)

---